# Subsections Issue

## Setup

### Imports

In [39]:
from typing import List
from pathlib import Path
from pprint import pprint
import re
import sys

import pandas as pd
import xlwings as xw

from buffered_iterator import BufferedIterator
import text_reader as tp
from sections import Rule, RuleSet, SectionBreak, ProcessingMethods, Section

### Logging

In [40]:
import logging
logging.basicConfig(format='%(name)-20s - %(levelname)s: %(message)s')
#logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger('Text Processing')
#logger.setLevel(logging.DEBUG)
logger.setLevel(logging.INFO)

### Initialize 2-line Section Tests

#### 2-line Section Source

In [41]:
GENERIC_TEST_TEXT = [
    'Text to be ignored',
    'StartSection Name: A',
    'EndSection Name: A',
    'StartSection Name: B',
    'EndSection Name: B',
    'More text to be ignored',
    ]

## Simple sections experimenting with start and end settings

### Two line section *StartSection*
- Start *Before* `StartSection`
- End *After* `EndSection`

In [42]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After')
    )

### Iterator Options

- If a regular iterator is supplied the *Step Back* item is lost
- The second call to `pprint(sub_section.read(test_iter))` returns an empty list 
  because the second *StartSection* line is lost.

In [43]:
test_iter = iter(GENERIC_TEST_TEXT)
pprint(sub_section.read(test_iter))

pprint(sub_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
[]


- Supplying a *BufferedIterator* preserves the *Step Back* item.
- The Third call to `pprint(sub_section.read(test_iter))` returns an empty list 
  because the default *aggregate* method is `list`, so no items in the sequence 
  returns an empty list.

In [44]:
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section.read(test_iter))
pprint(sub_section.read(test_iter))
pprint(sub_section.read(test_iter))

['StartSection Name: A', 'EndSection Name: A']
['StartSection Name: B', 'EndSection Name: B']
[]


- Using `subsections=sub_section` avoids the need for `BufferedIterator(GENERIC_TEST_TEXT)`

In [45]:
full_section = Section(
    section_name='Full',
    subsections=sub_section  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B'],
 []]


![Bad](../examples/error.png) Extra empty list at the end.

|Expected|Actual|
|-|-|
|`['StartSection Name: A', 'EndSection Name: A']`|`['StartSection Name: A', 'EndSection Name: A']`|
|`['StartSection Name: B', 'EndSection Name: B']`|`['StartSection Name: B', 'EndSection Name: B']`|
||`[]`|


### Single line section *StartSection*
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

In [46]:
sub_section1 = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before')
    )

test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))
pprint(sub_section1.read(test_iter))

['StartSection Name: A']
['StartSection Name: B']
[]


In [47]:

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section1]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A'], ['StartSection Name: B'], []]


![Bad](../examples/error.png) Extra empty list at the end.

|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['StartSection Name: A']`|
|`['StartSection Name: B']`|`['StartSection Name: B']`|
||`[]`|


### Single line section *EndSection*
#### Attempt #1
> - Start *Before* `EndSection`
> - End *Before* `StartSection`

- This doesn't work well because it includes the unwanted text after the last *EndSection*

In [48]:
sub_section2a = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))
pprint(sub_section2a.read(test_iter))

['EndSection Name: A']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|


#### Attempt #2
> - Start *Before* `EndSection`
> - End *After* `EndSection`

- This doesn't work because The first line is not tested so it only breaks after the second occurrence of *EndSection*.

In [49]:
sub_section2b = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))
pprint(sub_section2b.read(test_iter))

['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #3
> - Start *Before* `EndSection`
> - End *After* `EndSection`
> - Enable testing of first item

- This should work because the first line should be tested and trigger a break.

In [77]:
sub_section2c = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|


#### Attempt #3.5
> - Start *Before* `EndSection`
> - End *Before* `EndSection`
> - Enable testing of first item

- This should __<u>not</u>__ work because the first line should be tested and 
  trigger a break before returning anything.

In [78]:
sub_section2c = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))
pprint(sub_section2c.read(test_iter))

[]
[]
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`[]`|`[]`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Attempt #4
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Enable testing of first item

- This should work because the first line should be tested and always trigger a break.

In [79]:
sub_section2d = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))
pprint(sub_section2d.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['StartSection Name: A']`|`['EndSection Name: A']`|
|`['StartSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

#### Attempt #5
> - Start *Before* `EndSection`
> - End is `True` (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger on the second line.

In [80]:
sub_section2e = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))
pprint(sub_section2e.read(test_iter))

['EndSection Name: A', 'StartSection Name: B']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png) 

|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

**`end_section=SectionBreak(True)` defaults to `break_offset='Before'`**

#### Attempt #6
> - Start *Before* `EndSection`
> - End *After* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should <u>not</u> work because the break should trigger *After* the second line.

In [81]:
sub_section2f = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='After')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))
pprint(sub_section2f.read(test_iter))

['EndSection Name: A', 'StartSection Name: B']
['EndSection Name: B', 'More text to be ignored']
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', 'StartSection Name: B']`|`['EndSection Name: A', 'StartSection Name: B']`|
|`['EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|

#### Attempt #7
> - Start *Before* `EndSection`
> - Enable testing of first item
> - No End setting

- This should start with the first *EndSection* line and not stop until the end of the iterator.

In [82]:
sub_section2g = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_on_first_item=True,
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))
pprint(sub_section2g.read(test_iter))

['EndSection Name: A',
 'StartSection Name: B',
 'EndSection Name: B',
 'More text to be ignored']
[]
[]


![Good](../examples/Valid.png)
|Expected|Actual|
|-|-|
|`['EndSection Name: A', ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|`['EndSection Name: A', 'StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']`|
|`[]`|`[]`|
|`[]`|`[]`|


#### Single line section *EndSection* __Final Design__.
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

- This should work because the break should trigger *Before* the second line.

In [56]:
sub_section2 = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),
    end_section=SectionBreak(True, break_offset='Before')
    )
test_iter = BufferedIterator(GENERIC_TEST_TEXT)
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))
pprint(sub_section2.read(test_iter))

['EndSection Name: A']
['EndSection Name: B']
[]
[]


![Good](../examples/Valid.png) End breaks *Before* second line. 

|Expected|Actual|
|-|-|
|`['EndSection Name: A']`|`['EndSection Name: A']`|
|`['EndSection Name: B']`|`['EndSection Name: B']`|
|`[]`|`[]`|

### Combined Start and End subsections Single line section 
***sub_section1***
> - Start *Before* `StartSection`
> - End *Before* `EndSection`

***sub_section2*** 
> - Start *Before* `EndSection`
> - End *Before* ___`True`___ (Always Break)
> - Don't enable testing of first item

`subsections=[sub_section1, sub_section2]`

In [84]:

full_section = Section(
    section_name='Full',
    subsections=[sub_section1, sub_section2]
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A'], ['EndSection Name: A']],
 [['StartSection Name: B'], ['EndSection Name: B']],
 None]


![Bad](../examples/error.png) `None` as last item in *full_section* list.

|Expected|Actual|
|-|-|
|`['StartSection Name: A'], ['EndSection Name: A']`|`['StartSection Name: A'], ['EndSection Name: A']`|
|`['StartSection Name: B'], ['EndSection Name: B']`|`['StartSection Name: B'], ['EndSection Name: B']`|
||`None`|


# DONE TO HERE

**********

### Initial Section and Sub-Section Definitions

- Only definition line is:<br>
`subsections=[sub_section]`

In [58]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['Text to be ignored',
  'StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in all lines in one sub-list

**********

## Section Break options

### Add start to Section Definition

- Section start **Before** *StartSection*

`start_section=SectionBreak('StartSection', break_offset='Before'),`

In [59]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A',
  'EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Includes all lines after first *StartSection* in single sub-list.

### Add end to Section Definition

- Section start **Before** *StartSection*
- Section end **After** *EndSection*

```python
full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [60]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Includes both lines of first section in single sub-list.

### Define Muti-Section to Read Both Sections

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [61]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[[['StartSection Name: A', 'EndSection Name: A']],
 [['StartSection Name: B', 'EndSection Name: B']],
 [[]]]


- ![Bad](../examples/error.png) Includes both lines of each section in its own sub-list, but adds a blank sub-list at the end.

### Set Same Start and End Breaks for Section

Add end to Section Definition
- Section start **Before** *StartSection*
- Section end **After** *StartSection*
- Simple subsection
- Multi Section defines Full Section as Sub Section with no start or end (All lines)

```python
sub_section = Section(section_name='SubSection')

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before'),
    subsections=[sub_section] 
    )

multi_section = Section(
    section_name='Multi',
    subsections=[full_section] 
    )
```

In [62]:
full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('StartSection', break_offset='Before')
    )

multi_section = Section(section_name='Multi',
    subsections=[full_section] 
    )

pprint(multi_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A'],
 ['StartSection Name: B', 'EndSection Name: B', 'More text to be ignored']]


- Without SubSection, it works as expected.

- ![Bad](../examples/error.png) Appears to hang

**********

## SubSection Break Options

### Add *End on First* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_on_first_item=True,
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [63]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Bad](../examples/error.png) Results in `['EndSection Name: A']` sub-list, should have been `['StartSection Name: A']`.

### Add *End* as **True** to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection end is **True**  Should be the same as **End On First**

```python
sub_section = Section(section_name='SubSection',
    end_section=SectionBreak(True)
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [64]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A'], []]


## What should  `end_section=SectionBreak(True)` result in?

- ![Bad](../examples/error.png) Results in `['StartSection Name: A'], []` sub-lists, should have been `['StartSection Name: A']`.

### Add *Start* to Sub-Section

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [65]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

### Change Start Sub-Section to **After*

- Section start **Before** *StartSection*
- Section end **After** *EndSection*
- SubSection start **Before** *EndSection*

```python
sub_section = Section(section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before')
    )

full_section = Section(section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
```

In [66]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('StartSection', break_offset='After')
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )

pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/error.png) Results in blank sub-list.

### Add start to SubSection Definition
> `start_section=SectionBreak('EndSection', break_offset='Before')`

In [67]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    #start_section=SectionBreak('StartSection', break_offset='Before'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) Results in Empty list of lists

### Add *Start After* to Section Definition
> `start_section=SectionBreak('StartSection', break_offset='After')`

In [68]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before')`
    

In [69]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/error.png) Results in first and second section

### Add *Start After StartSection* to Section Definition and *Start Before End Section* to SubSection Definition and set *End On First Line* for SubSection
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_on_first_item=True,`
    

In [70]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A',
  'StartSection Name: B',
  'EndSection Name: B',
  'More text to be ignored']]


- ![Bad](../examples/Error.png) Results in second section

### Add *Start After StartSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After')`
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [71]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    #end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A'], ['EndSection Name: B'], []]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start After StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [72]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section] 
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to  *Before EndSection* and *End* to *`True` (Always Break)*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`
    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`
> `end_section=SectionBreak(True),`
    

In [73]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and for SubSection Definition, set *Start* to *Before EndSection*, *End* to *`True` (Always Break)* and *Keep Partial* to *`True`*
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
> 
> **SubSection Definition**<br>
> `start_section=SectionBreak('EndSection', break_offset='Before'),`<br>
> `end_section=SectionBreak(True),`<br>
> `keep_partial=True,`
    

In [74]:
sub_section = Section(
    section_name='SubSection',
    start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    keep_partial=True,
    end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[[]]


- ![Bad](../examples/Error.png) Results in empty list of lists

### Add *Start __Before__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='Before'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [75]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='Before'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['StartSection Name: A', 'EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in Two line section

### Add *Start __After__ StartSection* and *End Before EndSection* to Section Definition, and don't set any SectionBreaks for SubSection Definition, 
> **Section Definition**<br> 
> `start_section=SectionBreak('StartSection', break_offset='After'),`
> `end_section=SectionBreak('EndSection', break_offset='After'),`    
    

In [76]:
sub_section = Section(
    section_name='SubSection',
    #start_section=SectionBreak('EndSection', break_offset='Before'),  # Added to use alone
    #end_on_first_item=True,
    #keep_partial=True,
    #end_section=SectionBreak(True)
    )

full_section = Section(
    section_name='Full',
    start_section=SectionBreak('StartSection', break_offset='After'),
    end_section=SectionBreak('EndSection', break_offset='After'),
    subsections=[sub_section]  
    )
pprint(full_section.read(GENERIC_TEST_TEXT))

[['EndSection Name: A']]


- ![Good](../examples/Valid.png) Results in one line section